# Formatting Anki Flashcards

In this notebook, we are exploring the best way to prompt an LLM to improve the formatting of Anki flashcards. 

Approaches we are including:
1. Simple prompt
2. Simple prompt + Chain of Thought
3. Two-step process (critique → refine)
4. Agent

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
from anki.collection import Collection

from addon.application.use_cases.note_counter import is_note_marked_for_review
from addon.infrastructure.configuration.settings import AddonConfig

In [3]:
# Open an existing collection
col = Collection("/home/gianluca/.local/share/Anki2/User 1/collection.anki2")

# Do something with the collection
print(f"Number of notes: {col.note_count()}")
print(f"Number of cards: {col.card_count()}")

Number of notes: 3362
Number of cards: 3498


### Connect to Inference Server

For this notebook, we are going to use `Qwen3-14B-Q8_0.gguf` (from `unsloth`). This is a larger and more modern LLM compared to `Llama-3.1-8b-instruct`, which should lead to better results. We are also going to use the Chat Completion API, which 

In [4]:
from enum import Enum

class Mode(Enum):
    COMPLETION = "v1/completions"
    CHAT_COMPLETION = "v1/chat/completions"
    # response API not supported in vLLM yet

def answer(prompt, model_name: str = "./Qwen3-14B-Q8_0.gguf", mode: Mode = Mode.CHAT_COMPLETION):
    config = AddonConfig.create_nullable()

    url = f"http://iamgianluca.ddns.net:8080/{mode.value}"  # chat completion API
    payload = {
        "model": model_name,
        "messages": [  # TODO: parametrize. Should be "text" for Completion API
            # {'role': 'system', 'content': 'Answer directly without any thinking tags or explanations. /no_think'},
            {"role": "system", "content": prompt}
        ],
        "max_tokens": 4_000,  # config.max_tokens,
        "temperature": config.temperature,
        "top_p": config.top_p,
        "min_p": config.min_p,
        "top_k": config.top_k,
    }
    response = requests.post(url, json=payload)

    if mode == Mode.CHAT_COMPLETION:
        print(response)
        reasoning_content = response.json()["choices"][0]["message"][
            "reasoning_content"
        ]
        content = response.json()["choices"][0]["message"]["content"]  # chat completion API
        print(f"Content: {content.replace('<think>\n\n</think>\n\n', '')}")
        print(f"Reasoning: {reasoning_content}")
        return (content, reasoning_content)
        
    if mode == Mode.COMPLETION:
        print(response)
        content = response.json()["choices"][0]["text"]
        print(f"Content: {content}")
        return (content, None)

In [5]:
content, reasoning_content = answer(
    "Respond only with one word, lowercase, without punctuation. What is the Italian word for 'hello'? /no_think"
)

<Response [200]>
Content: ciao
Reasoning: None


We have everything we need now to tell the LLM to make some changes to our Anki flashcards.

Let's pull a few note currently marked for review.

In [6]:
deck_id = col.decks.current()["id"]
query = f"did:{deck_id}"
note_ids = col.find_notes(query)

NUM_NOTES_NEEDED = 10

flagged_notes = []
for note_id in note_ids:
    if is_note_marked_for_review(col, note_id):
        note = col.get_note(note_id)
        flagged_notes.append(note)

In [7]:
print(f"Number of flagged notes: {len(flagged_notes)}")

Number of flagged notes: 268


In [8]:
# col.close()

#### TODO

- [ ] Check if we have a class that we can use to read the collection from the hard disk and convert it to `AddonNote` instead of `Note`. In that way we can operate with domain objects instead of external dependencies. The same class should be used to convert the `AddonNote` back to `Note` (so maybe we should keep track of the `note_id`
- [ ] Once we have that class, we should update the rest of the codebase accordingly (e.g., note formatter, note counter, etc.)
- [ ] ...